# PEFT: LoRA

* LoRA 기반 PEFT를 학습하기 위한 예제 데이터셋으로 한국어 데이터셋인 koAlpaca의 100행만 추출하여 사용함

In [1]:
import pandas as pd

data_path = './datasets/alpaca_data.json'
df = pd.read_json(data_path)
df = df[['instruction', 'output']]
df = df.iloc[:100]
df.head()

,instruction,output
0,건강을 유지하기 위한 세 가지 팁을 알려주세요.,"세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는..."
1,세 가지 기본 색은 무엇인가요?,"기본 색은 빨강, 파랑, 노랑입니다."
2,원자의 구조를 설명하세요.,"원자는 양성자, 중성자, 전자로 구성되어 있으며, 양성자와 중성자는 원자핵 안에 있..."
3,대기 오염을 어떻게 줄일 수 있나요?,"대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기..."
4,어려운 결정을 내려야 했던 때를 설명하세요.,제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다....


* Pretrained LLM 들은 명시적인 Prompt 구조에서 입력을 해석하고 응답을 생성하도록 학습되어 있음.
* 때문에, 본격적인 모델 학습에 앞서 Llama3.1에서 지원하는 Instruct 구조에 맞게 데이터 형식을 수정해야 모델이 적절하게 문맥을 이해하여 안정적인 학습이 가능함.
* 각 데이터는 ‘user(사용자)’와 ‘assistant’(모델 응답)의 대화 형식으로 구성되어짐.

In [2]:
from datasets import Dataset

def format_example(row):
    return {
        'text': f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
        You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>
 
        {row['instruction']}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>
 
        {row['output']}<|eot_id|>"""
    }
 
# 판다스 데이터프레임을 데이터셋으로 변환하고, 포맷팅 함수 적용
dataset = Dataset.from_pandas(df)
dataset = dataset.map(format_example)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

* 사전학습된 모델을 불러오기 위해 Hugging Face에서 제공하는 AutoModelForCausalLM, AutoTokenizer를 import 
* AutoModelForCausalLM: 인과 언어 모델(주로 텍스트 생성에 쓰임)을 로드
* AutoTokenizer: 해당 모델에 맞는 토크나이저를 자동으로 로드

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# DeepSeek-R1-Distill-Llama-8B 모델 로드
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype="auto", device_map="auto")

print("DeepSeek-R1-Distill-Llama-8B 모델이 성공적으로 로드되었습니다.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

DeepSeek-R1-Distill-Llama-8B 모델이 성공적으로 로드되었습니다.


* 받은 모델을 간단히 테스트함. 질문을 tokenizer로 토큰으로 변환한 후 모델에 입력하고, 출력을 다시 tokenizer로 디코딩하여 모델 응답을 얻음
* 한국어 질문에 대해서 중국어로 답변을 내놓으므로 정확한 답이 될수 없음. 이문제를 LoRA로 해결해봄

In [3]:
from transformers import pipeline
 
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=256)
 
def generate_and_stop(prompt):
    result = pipe(f"{prompt}")[0]['generated_text']
    return result
 
prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>
 
건강을 유지하기 위한 세 가지 팁을 알려주세요.<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>
"""
 
print(generate_and_stop(prompt))

Device set to use cuda:0


<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
You are a helpful assistant<|eot_id|>
<|start_header_id|>user<|end_header_id|>
 
건강을 유지하기 위한 세 가지 팁을 알려주세요.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
嗯，用户让我给出三个保持健康的建议。好的，首先得考虑用户的需求，可能他们想知道如何更好地维护自己的身体和心理健康。保持健康很重要，所以我要提供实用的建议。

首先，饮食方面肯定是关键。建议用户多吃蔬菜和水果，保持均衡的营养。这能帮助他们获得必要的维生素和矿物质，增强免疫力。然后，运动也很重要，建议每周至少做几次有氧运动，比如跑步或游泳，这样有助于保持心肺功能健康。还有，强调定期体检，早期发现问题，预防疾病，这也是很重要的。

另外，心理健康也不能忽视。建议用户学会放松，比如冥想或深呼吸，这样能缓解压力。保持良好的人际关系也很重要，帮助他们建立一个支持性的社交网络。最后，充足


* PEFT의 파라미터를 설정 
* LoraConfig()를 통해서 LoRA 모델의 특정 파라미터만 조정하여 미세 조정(PEFT)을 수행
* lora_dropout은 과적합을 방지하기 위해 레이어의 가중치를 드롭아웃할 확률
* prepare_model_for_kbit_training을 통해 메모리를 절약하면서도 성능을 유지

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_params = LoraConfig(
    lora_alpha=16, #scaling factor
    lora_dropout=0.05, #과적합 방지
    r=16, #low-rank 차원 수
    bias="none", #bias 항에 대해서는 학습을 하지 않음
    task_type="CAUSAL_LM", #Causal Language Model
    target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj'] #LoRA/DoRA가 적용될 대상 레이어
)

model = prepare_model_for_kbit_training(model, 8) #model을 8 bit 양자화 상태에서 학습 가능하도록 준비
model = get_peft_model(model, peft_params) #8bit model에 대해 PEFT 버전 모델로 변환

* 모델 학습 관련 파라미터들을 설정하기 위해 transformers 라이브러리의 TrainingArguments 클래스를 import
* 이 클래스는 Trainer 또는 SFTTrainer (Supervised Fine-Tuning Trainer) 와 같은 학습 클래스에 전달되어 학습 전반을 제어

In [5]:
from transformers import TrainingArguments 

training_params = TrainingArguments(
    output_dir="./results", #학습된 모델 체크포인트, config, tokenizer 등을 저장할 디렉토리 경로
    logging_dir="./logs", #학습 과정의 손실, 정확도, 학습률 등의 로그를 저장할 디렉토리 경로
    logging_steps=10, #몇 스텝마다 로그를 저장할지 설정
    save_steps=1000, #몇 스텝마다 모델을 저장할지 설정
    num_train_epochs=50, #전체 학습 데이터셋을 몇 번 반복(epoch)할지를 지정                   
    per_device_train_batch_size=1, #GPU(혹은 CPU)가 한 번에 처리할 샘플의 수       
    gradient_accumulation_steps=8, #8번의 스탭마다 1번 파라미터를 업데이트
    learning_rate=2e-4, #학습률이 0.0002              
    fp16=True, #학습 시 일부 연산을 16 bit로 수행하여 메모리 절약 (원래는 32 bit)          
)

* SFTTrainer를 사용해 학습 설정
    * train_dataset: 보통 JSON 형태로 prompt + response 를 합쳐 text로 구성
    * peft_config: LoRA 적용 시 사용


In [6]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model, #학습시킬 모델
    train_dataset=dataset, #학습에 사용할 데이터셋
    args=training_params, #학습 설정 객체(TrainingArguments)
    peft_config=peft_params, #PEFT 관련 설정 객체(LoraConfig)
)

Converting train dataset to ChatML:   0%|          | 0/100 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


* 학습 파라미터인 num_train_epochs는 50인데, 배치 사이즈(한 스텝에 처리하는 샘플 수)에 따라 600 Step으로 쪼개어짐 
* 50 Epoch (한 데이터를 50번 반복해서 학습) 한 결과 Loss 값이 4에서 (초기 10 Step)에서 0.31(최종 600 Step)까지 감소함

In [7]:
trainer.train()

Step,Training Loss
10,4.043500
20,2.344800
30,1.924500
40,1.761600
50,1.420000
60,1.346000
70,1.083400
80,1.077600
90,0.873400
100,0.791100


TrainOutput(global_step=600, training_loss=0.5788791978359222, metrics={'train_runtime': 26139.4439, 'train_samples_per_second': 0.191, 'train_steps_per_second': 0.023, 'total_flos': 1.792310927806464e+16, 'train_loss': 0.5788791978359222})

In [2]:
# 6. 모델 저장
model.save_pretrained("./Deepseek-R1-Distill-Llama-8b_LoRA_alpace_50")
tokenizer.save_pretrained("./Deepseek-R1-Distill-Llama-8b_LoRA_alpace_50")

('./Deepseek-R1-Distill-Llama-8b_LoRA_alpace_50\\tokenizer_config.json',
 './Deepseek-R1-Distill-Llama-8b_LoRA_alpace_50\\special_tokens_map.json',
 './Deepseek-R1-Distill-Llama-8b_LoRA_alpace_50\\tokenizer.json')

* 미세조정된 모델은 같은 한국어 질문에 대해서 중국어가 아닌 한국어로 답변했고, 그 결과가 학습 데이터와 유사하게 나타남
* 하지만, 문장 속의 표현이 어색하고 어순이 잘못되어 있음 

In [2]:
from transformers import pipeline
 
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=256)
 
def generate_and_stop(prompt):
    result = pipe(f"{prompt}")[0]['generated_text']
    return result
 
prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>
 
건강을 유지하기 위한 세 가지 팁을 알려주세요.<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>
"""
 
print(generate_and_stop(prompt))

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
You are a helpful assistant<|eot_id|>
<|start_header_id|>user<|end_header_id|>
 
건강을 유지하기 위한 세 가지 팁을 알려주세요.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

세 가지 팁 중 하나는 식습관을 지키는 건강을 유지하기 위한 중요한 요소입니다. 두 번째 팁은 규칙적으로 수면을 유지하며 세 번째 팁은 적극적으로 운동을 하는 것입니다.


# PEFT: DoRA

* 기존 LoRA의 학습 코드에서  LoraConfig() 함수 속에 use_dora 매개변수만 추가하여 use_dora=True 꼴로 작성해주면, LoRA 대신 DoRA를 사용할 수 있음

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_params = LoraConfig(
    use_dora=True, #LoRA가 아닌 DoRA를 사용함.
    lora_alpha=16, #scaling factor
    lora_dropout=0.05, #과적합 방지
    r=16, #low-rank 차원 수
    bias="none", #bias 항에 대해서는 학습을 하지 않음
    task_type="CAUSAL_LM", #Causal Language Model
    target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj'] #LoRA/DoRA가 적용될 대상 레이어
)

model = prepare_model_for_kbit_training(model, 8) #model을 8 bit 양자화 상태에서 학습 가능하도록 준비
model = get_peft_model(model, peft_params) #8bit model에 대해 PEFT 버전 모델로 변환

* LoRA의 학습 결과와 비교했을 때, LoRA로 미세조정한 LLM의 응답은 어색한 문장 구조를 나타내고 있지만, DoRA로 미세조정한 LLM의 응답은 학습 데이터와 거의 일치하는 출력 결과를 나타냄

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

model_path = "Deepseek-R1-Distill-Llama-8b_LoRA_alpace_50"

peft_config = PeftConfig.from_pretrained(model_path)

# 2. Base 모델 로드 (LoRA를 적용할 원본 모델)
base_model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path)

# 3. LoRA 가중치를 base model에 적용
model = PeftModel.from_pretrained(base_model, model_path)

# 4. Tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from transformers import pipeline
 
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=256)
 
def generate_and_stop(prompt):
    result = pipe(f"{prompt}")[0]['generated_text']
    return result
 
prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>
 
건강을 유지하기 위한 세 가지 팁을 알려주세요.<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>
"""
 
print(generate_and_stop(prompt))

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
You are a helpful assistant<|eot_id|>
<|start_header_id|>user<|end_header_id|>
 
건강을 유지하기 위한 세 가지 팁을 알려주세요.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

 세 가지 팁 중 하나는 세면을 시간적으로 관리하는 방법이입니다. 두 번째 팁은 꾸준한 수면이 건 건강에 큰 영향을 미치도록 있습니다. 세 번째 팁은 적절한 식습관을 유지하는 방법이입니다.


In [3]:
from transformers import pipeline
 
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=256)
 
def generate_and_stop(prompt):
    result = pipe(f"{prompt}")[0]['generated_text']
    return result
 
prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>
 
대기 오염을 어떻게 줄일 수 있나요?<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>
"""
 
print(generate_and_stop(prompt))

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
You are a helpful assistant<|eot_id|>
<|start_header_id|>user<|end_header_id|>
 
대기 오염을 어떻게 줄일 수 있나요?<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
 줄이는 대로 도rive하고, 대기 오염 방지를 위해 바다에 폐기물을 던지하지 않고 처리하고, 전기 발전소를 미세먼지로 효율이 고쳐져는 등 대기 오염을 줄이기 위해 고쳐져 있습니다.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

model_path = "Deepseek-R1-Distill-Llama-8b_DoRA_alpaca_50"

peft_config = PeftConfig.from_pretrained(model_path)

# 2. Base 모델 로드 (LoRA를 적용할 원본 모델)
base_model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path)

# 3. LoRA 가중치를 base model에 적용
model = PeftModel.from_pretrained(base_model, model_path)

# 4. Tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained(model_path)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
from transformers import pipeline
 
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=256)
 
def generate_and_stop(prompt):
    result = pipe(f"{prompt}")[0]['generated_text']
    return result
 
prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>
 
건강을 유지하기 위한 세 가지 팁을 알려주세요.<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>
"""
 
print(generate_and_stop(prompt))

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
You are a helpful assistant<|eot_id|>
<|start_header_id|>user<|end_header_id|>
 
건강을 유지하기 위한 세 가지 팁을 알려주세요.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

    세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.


In [3]:
from transformers import pipeline
 
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=256)
 
def generate_and_stop(prompt):
    result = pipe(f"{prompt}")[0]['generated_text']
    return result
 
prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>
 
대기 오염을 어떻게 줄일 수 있나요?<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>
"""
 
print(generate_and_stop(prompt))

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 
You are a helpful assistant<|eot_id|>
<|start_header_id|>user<|end_header_id|>
 
대기 오염을 어떻게 줄일 수 있나요?<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
assistant
 
대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기 위해 차량 정기 검사를 받아야 합니다. 또한 에너지 절약을 위해 전기를 아끼고, 나무를 심어 대기 오염을 줄일 수 있습니다.
